## 10 Bit Lookup-Table für DAC 100




Wichmann, A 2025-08-24

Schaltungsaufbauf

<img src="DAC100-schaltplan.png" alt="text" />

In [1]:
# Import time: Time access and conversions to allow for pausing
import time
import numpy as np
import scpi
import math




In [2]:
# Define RedPitaya SCPI Object
# Nur zum testen, ist erstmal nicht notwendig
# redPitaty = scpi.redpitaya('192.168.178.98', 5000)

# Define DP832A SCPI Object (Power Supply)
dac100 = scpi.DAC100("192.168.178.111", 5060)

# Define DMM6500 6 1/2 Digit SCPI Object (DMM)
dmm6500 = scpi.DMM6500( "192.168.178.150", 5025)

# Display ausschalten
# dmm6500.setDisplayOff()



# Initialisierung des Messsystems

In [3]:
#setup keithley dmm6500
idn = dmm6500.getIdent()

print( f"DMM: {idn}" ) 
dmm6500.reset()
dmm6500.setFunctionVoltDC()
# dmm6500.setRange("AUTO")
dmm6500.setNPLC(10) 
v1 = dmm6500.getVoltDC()
print( f"DMM: {v1:2.5f}" ) 




DMM: KEITHLEY INSTRUMENTS,MODEL DMM6500,04461523,1.0.04b

DMM: 10.00004


# DAC Bereich durchlaufen

In [4]:
dac100.setDACA( 65535 // 2 ) # Grob
dac100.setDACB( 65535 // 2 ) # Feine


for i in range(-10, 11):  # Ganzzahlen von -10 bis 10
    x = i * 1.0           # Umwandlung in float
    
    dac100.setVoltage( x );
    v1 = dmm6500.getVoltDC()
   
    print(f"DAC10: {x:2.5f} | DMM: {v1:2.5f}")



DAC10: -10.00000 | DMM: -9.99997
DAC10: -9.00000 | DMM: -8.99998
DAC10: -8.00000 | DMM: -7.99999
DAC10: -7.00000 | DMM: -6.99998
DAC10: -6.00000 | DMM: -5.99996
DAC10: -5.00000 | DMM: -4.99997
DAC10: -4.00000 | DMM: -4.00001
DAC10: -3.00000 | DMM: -2.99999
DAC10: -2.00000 | DMM: -2.00000
DAC10: -1.00000 | DMM: -1.00000
DAC10: 0.00000 | DMM: 0.00004
DAC10: 1.00000 | DMM: 1.00004
DAC10: 2.00000 | DMM: 2.00000
DAC10: 3.00000 | DMM: 3.00000
DAC10: 4.00000 | DMM: 4.00001
DAC10: 5.00000 | DMM: 5.00000
DAC10: 6.00000 | DMM: 6.00002
DAC10: 7.00000 | DMM: 7.00002
DAC10: 8.00000 | DMM: 8.00003
DAC10: 9.00000 | DMM: 9.00000
DAC10: 10.00000 | DMM: 10.00002


In [23]:


##### Diagrammerstellung
output_notebook # Ausgabe direkt in Jupyter

#for item in diffList:
#    diffList[item = item - 2.6

# create a new plot with a title and axis labels
p = figure(title="Abweichung vom idealen Wert", plot_height=600, plot_width=900)

p.xaxis.axis_label = 'Step'
p.yaxis.axis_label = 'Voltage in mV'
# add a line renderer with legend and line thickness
p.line(stepList, diffList, line_width=1, color="blue")
#p.line(realV, dutV, line_width=1, color="green")


# show the results
show(p, notebook_handle=True)

print( "%2.6f;%2.6f;%2.6f" % ( realV[10], dutV[10] ,adj(dutV[10])))


1.006307;1.007024;1.005550


In [14]:

def adj(vCalc):
    p1_xr = 0.00576;
    p1_yc = 0.006516;
    p2_xr = 5.006949;
    p2_yc = 5.0113;
    
    m = ( p2_yc - p1_yc ) / (p2_xr - p1_xr);
  
    b =  p1_yc - ( m * p1_xr);

    adj = (vCalc - b ) / m;
    
    return adj